In [1]:
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders

In [2]:
import pyalex

pyalex.config.email = "b.cliff@gwmail.gwu.edu"

In [15]:
import json

#### Getting the list of entities

In [3]:
works = Works().get()
topics = Topics().get()
authors = Authors().get()
institutions = Institutions().get()

In [6]:
# Picking random entry from works
works[0]

{'id': 'https://openalex.org/W3038568908',
 'doi': 'https://doi.org/10.1585/pfr.15.2402039',
 'title': 'Radiation Resistant Camera System for Monitoring Deuterium Plasma Discharges in the Large Helical Device',
 'display_name': 'Radiation Resistant Camera System for Monitoring Deuterium Plasma Discharges in the Large Helical Device',
 'publication_year': 2020,
 'publication_date': '2020-06-08',
 'ids': {'openalex': 'https://openalex.org/W3038568908',
  'doi': 'https://doi.org/10.1585/pfr.15.2402039',
  'mag': '3038568908'},
 'language': 'en',
 'primary_location': {'id': 'doi:10.1585/pfr.15.2402039',
  'is_oa': True,
  'landing_page_url': 'https://doi.org/10.1585/pfr.15.2402039',
  'pdf_url': 'https://www.jstage.jst.go.jp/article/pfr/15/0/15_2402039/_pdf',
  'source': {'id': 'https://openalex.org/S46033839',
   'display_name': 'Plasma and Fusion Research',
   'issn_l': '1880-6821',
   'issn': ['1880-6821'],
   'is_oa': True,
   'is_in_doaj': False,
   'is_core': True,
   'host_organizat

In [7]:
works[0].keys()

dict_keys(['id', 'doi', 'title', 'display_name', 'publication_year', 'publication_date', 'ids', 'language', 'primary_location', 'type', 'indexed_in', 'open_access', 'authorships', 'institutions', 'countries_distinct_count', 'institutions_distinct_count', 'corresponding_author_ids', 'corresponding_institution_ids', 'apc_list', 'apc_paid', 'fwci', 'has_fulltext', 'cited_by_count', 'citation_normalized_percentile', 'cited_by_percentile_year', 'biblio', 'is_retracted', 'is_paratext', 'is_xpac', 'primary_topic', 'topics', 'keywords', 'concepts', 'mesh', 'locations_count', 'locations', 'best_oa_location', 'sustainable_development_goals', 'grants', 'awards', 'funders', 'has_content', 'referenced_works_count', 'referenced_works', 'related_works', 'abstract_inverted_index', 'counts_by_year', 'updated_date', 'created_date'])

In [ ]:
# Picking a related award
Works()['W2889162861']

#### Filtering Works Records

In [ ]:
Works().filter(publication_year=2020, is_oa=True).get()

In [ ]:
# Filtering works from Japan, France, Germany, and England
country_codes = ['jp', 'fr', 'de', 'gb']
Works().filter(
    institutions={'country_code':country_codes}
).get()

#### Playing with the Search Filter function

In [ ]:
ai_funders = Funders().search_filter(display_name='artificial intelligence').get()

### Cleaning data

In [80]:
works_filter = Works().filter(publication_year=2024, is_oa=True).select(['id', 'doi', 'title',
'publication_date', 'type', 'related_works']).get()

In [81]:
works_filter[0]

{'id': 'https://openalex.org/W4392145873',
 'doi': 'https://doi.org/10.3929/ethz-b-000667478',
 'title': 'Two new Later Stone Age sites from the Final Pleistocene in the Falémé Valley, eastern Senegal',
 'publication_date': '2024-03-28',
 'type': 'preprint',
 'related_works': ['https://openalex.org/W1988219939',
  'https://openalex.org/W1980130043',
  'https://openalex.org/W2468743120',
  'https://openalex.org/W2417572233',
  'https://openalex.org/W2040370458',
  'https://openalex.org/W2471004180',
  'https://openalex.org/W2401598822',
  'https://openalex.org/W65102778',
  'https://openalex.org/W2927808016',
  'https://openalex.org/W2068776971']}

In [82]:
cleaned_ids = [work.get('id').replace('https://openalex.org/', '') for work in works_filter]
cleaned_related_works = [[rel.replace('https://openalex.org/', '') for rel in work['related_works']] for work in works_filter]

In [83]:
for rec, clean_id, clean_rel_id in zip(works_filter, cleaned_ids, cleaned_related_works):
    rec['id'] = clean_id
    rec['related_works'] = clean_rel_id

In [90]:
filtered_abstracts = [Works()[work.get('id')]['abstract'] for work in works_filter]

In [96]:
filtered_abstracts

['Kenya one of the African countries has pledged to reduce neonatal death as per the 2030 World Health Organization target. Providing high-quality newborn care is critical in minimizing neonatal mortality. This study aimed to determine the factors that influence the quality of newborn care in Kenya. Secondary data from 11,863 participants of the 2022 Kenya Demographic and Health Survey (KDHS) were analyzed. The participants were chosen using two-stage stratified sampling. The quality of newborn care was operationalized as receiving all components of newborn care after childbirth, as reported by the mother. Using SPSS (version 29), univariate and multivariable logistic regression analyses were used to analyse the data. In this study, 32.7% (95% confidence interval [CI]: 31.0%-34.5%) of the mothers reported that their newborns had received all components of quality neonatal care after childbirth. Mothers who spent an average of one hour accessing the health facilities compared with those

In [97]:
for rec, abs in zip(works_filter, filtered_abstracts):
    rec['pull_abs'] = abs if not None else ''

In [98]:
works_filter

KeyError: 'abstract_inverted_index'

#### Including Filter and Select

In [25]:
# Determining the keys for each Works() object
Works()['W2889162861']['abstract']

'The TRIGA Mark-III nuclear reactor has several intrinsic security specifications related with the reactor core and a biological shield provided by the bathtub itself. However, to shield the exposition to gamma rays and thermal neutrons is vital for protecting the workers exposed to ionizing radiation or the incorporation of radioactive material. For this reason, in this work we present a study for designing the shield for the neutron diffractometer of the nuclear reactor, with sufficient thickness to diminish the neutron flux (4.8x108 n/cm2-s), and simultaneously to allow the reduction of the dose speed. (Correlation between the absorbed dose and the probability that the stochastic effects appear). For considering the shield dimensions it is necessary to select the adequate materials which allow the attenuation of the gamma radiation and the neutron flux in the peripheral of the diffractometer. The shield design proposed is constituted by a polythene mixture, paraffin and boron, (PBP)

In [79]:
with open('./data/works_24_filtered.json', 'w') as f:
    f.write(json.dumps(works_filter))